<a href="https://colab.research.google.com/github/nmonson1/mnist_exploration/blob/main/MNIST_hessian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import math

import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from PIL import Image

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 1, 3, 1)
    self.fc2 = nn.Linear(169, 10)
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
      module.weight.data.normal_(mean=0.0, std=0.5)
      if module.bias is not None:
        module.bias.data.zero_()

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = torch.flatten(x, 1)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

In [ ]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
EPOCHS = 10
LR = 1.0
GAMMA = 0.7
num_params = 1710

In [ ]:
use_cuda = torch.cuda.is_available()

if use_cuda:
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [ ]:
model = Net().to(DEVICE)
optimizer = optim.Adadelta(model.parameters(), lr=LR)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
train_kwargs = {'batch_size': BATCH_SIZE}
test_kwargs = {'batch_size': TEST_BATCH_SIZE}
if use_cuda:
  cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
  train_kwargs.update(cuda_kwargs)
  test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(Subset(dataset1, np.random.randint(len(dataset1), size=1500)), **train_kwargs)  # subset of datasets to keep network overparameterized
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 88174176.27it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 16666991.45it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26185251.05it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11319387.27it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
def train(dry_run: bool, model, device, train_loader, optimizer, epoch):
  model.train()

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)

    loss.backward()
    optimizer.step()

    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
      if dry_run:
        break


def test(model, device, test_loader):
  model.eval()

  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
      pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [ ]:
scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
  train(False, model, DEVICE, train_loader, optimizer, epoch)
  test(model, DEVICE, test_loader)
  scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_small.pt")

Train Epoch: 1 [0/1500 (0%)]	Loss: 34.024174
Train Epoch: 1 [640/1500 (42%)]	Loss: 20.776669
Train Epoch: 1 [1280/1500 (83%)]	Loss: 14.220222

Test set: Average loss: 13.6152, Accuracy: 1240/10000 (12%)

Train Epoch: 2 [0/1500 (0%)]	Loss: 14.740343
Train Epoch: 2 [640/1500 (42%)]	Loss: 11.231129
Train Epoch: 2 [1280/1500 (83%)]	Loss: 8.936780

Test set: Average loss: 8.9646, Accuracy: 1498/10000 (15%)

Train Epoch: 3 [0/1500 (0%)]	Loss: 9.846889
Train Epoch: 3 [640/1500 (42%)]	Loss: 8.023938
Train Epoch: 3 [1280/1500 (83%)]	Loss: 6.735827

Test set: Average loss: 6.9281, Accuracy: 1711/10000 (17%)

Train Epoch: 4 [0/1500 (0%)]	Loss: 7.620111
Train Epoch: 4 [640/1500 (42%)]	Loss: 6.407550
Train Epoch: 4 [1280/1500 (83%)]	Loss: 5.571795

Test set: Average loss: 5.8107, Accuracy: 1870/10000 (19%)

Train Epoch: 5 [0/1500 (0%)]	Loss: 6.363405
Train Epoch: 5 [640/1500 (42%)]	Loss: 5.472382
Train Epoch: 5 [1280/1500 (83%)]	Loss: 4.874392

Test set: Average loss: 5.1312, Accuracy: 1994/10000 (

In [ ]:
def get_hessian(model: nn.Module, loss: torch.Tensor):
  # Hessian of the loss w.r.t. the parameters

  optimizer = optim.Adadelta(model.parameters(), lr=1)
  model = model.to(DEVICE)
  model.eval()
  optimizer.zero_grad()

  xs = optimizer.param_groups[0]['params']
  ys = loss

  grads = torch.autograd.grad(ys.to(DEVICE), xs, create_graph=True)

  grads2 = []
  for j, grad in enumerate(grads):
    start = time.time()
    print(f"Calculating pairwise (every param, every layer) second-order derivatives for params of layer {j}...")

    grad = torch.reshape(grad, [-1])

    grads2_tmp = []
    for count, g in enumerate(grad):
      percent_done = count / (len(grad))
      if (count % 100 == 0):
        print(f"{percent_done:.2%} done...")

      grads2_tmp_tmp = []  # creating variable naming, innit?
      for x in xs:
        g2 = torch.autograd.grad(g, x, retain_graph=True)[0]
        g2 = torch.reshape(g2, [-1])
        grads2_tmp_tmp.append(g2.data.cpu())

      flat = None
      for second_orders in grads2_tmp_tmp:
        if(flat is None):
          flat = second_orders.flatten()
        else:
          flat = torch.concat((flat, second_orders.flatten()))

      grads2_tmp.append(flat.numpy())
    grads2 += grads2_tmp

    print('Time used is ', time.time() - start)
  return np.array(grads2)

In [ ]:
losses = []
for i, (data, targets) in enumerate(train_loader):
  print(f"Computing average loss on train set {(i+1)/len(train_loader):.2%}   ", end='\r')
  data = data.to(DEVICE)
  targets = targets.to(DEVICE)

  losses.append(loss_fn(model(data), targets))

avg_loss = torch.mean(torch.stack(losses))
print(f'Average Train Loss: {avg_loss.item()} \n')

hessian = get_hessian(model, avg_loss)

Average Train Loss: 3.8263282775878906 

Calculating pairwise (every param, every layer) second-order derivatives for params of layer 0...
0.00% done...
Time used is  3.1911449432373047
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 1...
0.00% done...
Time used is  0.11049914360046387
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 2...
0.00% done...
5.92% done...
11.83% done...
17.75% done...
23.67% done...
29.59% done...
35.50% done...
41.42% done...
47.34% done...
53.25% done...
59.17% done...
65.09% done...
71.01% done...
76.92% done...
82.84% done...
88.76% done...
94.67% done...
Time used is  243.7737901210785
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 3...
0.00% done...
Time used is  0.9884436130523682


In [ ]:
hessian.shape

(1710, 1710)